# Trigger efficiency study

In [ ]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import exists
from typing import Union, List
from hist import Hist
from hist.intervals import clopper_pearson_interval
import vector

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import hist
import mplhep as hep

hep.style.use(["CMS", "firamath"])

formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 12})
plt.rcParams["lines.linewidth"] = 2
plt.rcParams["grid.color"] = "#CCCCCC"
plt.rcParams["grid.linewidth"] = 0.5
plt.rcParams["figure.edgecolor"] = "none"


import sys

In [ ]:

def check_selector(sample: str, selector: Union[str, List[str]]):
    if isinstance(selector, list) or isinstance(selector, tuple):     # If selector is a list or tuple
        
        for s in selector:     # Iterate over the list or tuple
            if s.startswith("*"):     # If selector starts with '*', check if the rest of the selector is in sample
                if s[1:] in sample:
                    return True
            else:
                if sample.startswith(s):
                    return True
    else:   # If selector is a single string
        if selector.startswith("*"):
            if selector[1:] in sample:
                return True
        else:
            if sample.startswith(selector):
                return True

    return False

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

#### Reading parquets for the samples

In [ ]:
data_dir = "/Users/gbibim/Here/genZ"
y = "2022"
samples = {
    "2022": {
        "Zto2Q-4Jets": ["Zto2Q-4Jets_HT-400to600", "Zto2Q-4Jets_HT-600to800", "Zto2Q-4Jets_HT-800", "Zto2Q-4Jets_HT-200to400"],
        "Zto2Q-2Jets": ["Zto2Q-2Jets_PTQQ-100to200_1J", "Zto2Q-2Jets_PTQQ-200to400_1J", "Zto2Q-2Jets_PTQQ-400to600_1J", 
                        "Zto2Q-2Jets_PTQQ-600_1J", "Zto2Q-2Jets_PTQQ-100to200_2J", "Zto2Q-2Jets_PTQQ-200to400_2J", "Zto2Q-2Jets_PTQQ-400to600_2J", "Zto2Q-2Jets_PTQQ-600_2J" ],
    },
}[y]  #This is a way to retrieve the dictionary for the year "2022" from the dictionary of samples


full_samples_list = listdir(f"{data_dir}/{y}/") # list of all samples in the directory

events_dict = {}


##### If we want all the different HT and PtQQ samples together

In [ ]:
for label, selector in samples.items():  #Iterating over the dictionary of samples. Label is the key and selector is the value associated with the key
    #label = Zto2Q-4Jets
    #selector = Zto2Q-4Jets_HT-400to600, Zto2Q-4Jets_HT-200to400, Zto2Q-4Jets_HT-600

    events_dict[label] = {}  # initiates an empty list for the label key in the events_dict dictionary
    print(selector)
    
    for sample in selector:
        events_dict[label][sample] = [] 
        if not check_selector(sample, selector):    # checks if the sample is in the selector list
            print(sample)
            print("HERE")
            continue
        if not exists(f"{data_dir}/{y}/{sample}/parquet"):    #checks if the parquet file exists
            print(f"No parquet file for {sample}")
            continue


        events = pd.read_parquet(f"{data_dir}/{y}/{sample}/parquet", columns=None)
        not_empty = len(events) > 0
        if not_empty:
            events_dict[label][sample] = events
        print(f"Loaded {sample: <50}: {len(events)} entries")


    # If there are any DataFrames for the current label, concatenate them
    #if events_dict[label]:
    #    all_events = pd.concat(events_dict[label].values(), ignore_index=True)
    #    events_dict[label]['all'] = all_events  # Store the concatenated DataFrame under a special key

    # If no data was loaded for the label, delete the entry
    if not events_dict[label]:
        del events_dict[label]




In [ ]:
print(events_dict.keys())

In [ ]:
print(events_dict["Zto2Q-4Jets"]["Zto2Q-4Jets_HT-400to600"])

In [ ]:
trigger_dict = {
    "Combined": (
        [
            "AK8PFJet230_SoftDropMass40",
        ],
        "label",
    ),
}

In [ ]:
# ht_bins_fine = (25, 200, 3000)
ht_bins_fine = (25, 200, 2000)
mhh_axis = hist.axis.Regular(40, 250, 1500, name="mhh", label=r"$m_{HH}$ [GeV]")
ht_axis = hist.axis.Regular(*ht_bins_fine, name="ht", label=r"HT [GeV]")
fjpt_axis = hist.axis.Regular(10, 250, 600, name="pt", label=r"fj pT$^0$ [GeV]")
cat_axis = hist.axis.StrCategory([], name="cat", growth=True)
sample_axis = hist.axis.StrCategory([], name="sample_name", growth=True)

hht = hist.Hist(ht_axis, cat_axis, sample_axis)
hmhh = hist.Hist(mhh_axis, cat_axis, sample_axis)
hpt = hist.Hist(fjpt_axis, cat_axis, sample_axis)

for label in events_dict.keys():
    for sample, events in events_dict[label].items():
        ht = sample["ht"][0]
        pt_0 = events["ak8FatJetPt"][0]
        gen_higgs = make_vector(events, "GenHiggs")
        msd_0 = events["ak8FatJetMsd"][0]
        mreg_0 = events["ak8FatJetPNetMass"][0]

        mhh = (gen_higgs[:, 0] + gen_higgs[:, 1]).m

        selection = pt_0 > 200
        sel_pt = (pt_0 > 200) & ((msd_0 > 40) | (mreg_0 > 40))

        hht.fill(ht=ht, cat="denominator", sample_name=sample)
        hmhh.fill(mhh=mhh, cat="denominator", sample_name=sample)
        hpt.fill(pt=pt_0[sel_pt], cat="denominator", sample_name=sample)
        for trigger_title, (triggers, trigger_label) in trigger_dict.items():
            trigger_selection = np.zeros_like(selection)
            for hlt in triggers:
                trigger_selection |= (events[hlt].values == 1).squeeze()
            num_selection = trigger_selection
            hht.fill(ht=ht[num_selection], cat=trigger_title, sample_name=sample)
            hmhh.fill(mhh=mhh[num_selection], cat=trigger_title, sample_name=sample)
            hpt.fill(pt=pt_0[sel_pt & num_selection], cat=trigger_title, sample_name=sample)
    

In [ ]:
ht_bins_fine = (25, 200, 2000)
zpt_axis = hist.axis.Regular(100, 0, 1000, name="pt", label=r"Z pT$^0$ [GeV]") #!!!
fjpt_axis = hist.axis.Regular(100, 200, 1000, name="pt", label=r"fj pT$^0$ [GeV]")
cat_axis = hist.axis.StrCategory([], name="cat", growth=True)
sample_axis = hist.axis.StrCategory([], name="sample_name", growth=True)

hpt_fj = hist.Hist(fjpt_axis, cat_axis, sample_axis)
hpt_z = hist.Hist(zpt_axis, cat_axis, sample_axis)

for label, sample_dict in events_dict.items():
    for sample, events in sample_dict.items():
        #events = events[0]
        pt_z = events["GenVPt"][0]
        pt_fj = events["ak8FatJetPt"][0]
        SDm = events["ak8FatJetmsoftdrop"][0]
        wd = events["weight"][0]
        #selection = (pt_0 > 0)
        selection = SDm > 40


        hpt_fj.fill(pt=pt_fj[selection], cat="denominator", sample_name=sample, weight=wd[selection])
        hpt_z.fill(pt=pt_z[selection], cat="denominator", sample_name=sample, weight=wd[selection])


        hpt_fj.fill(pt=pt_fj[selection], cat="denominator", sample_name="todas")
        hpt_z.fill(pt=pt_z[selection], cat="denominator", sample_name="todas")

        # Fill histograms for each trigger category
        for trigger_title, (triggers, trigger_label) in trigger_dict.items():
            trigger_selection = np.zeros_like(selection)
            for hlt in triggers:
                trigger_selection |= (events[hlt].values == 1).squeeze()
            num_selection = trigger_selection & selection

            hpt_fj.fill(pt=pt_fj[num_selection], cat=trigger_title, sample_name=sample, weight=wd[num_selection])
            hpt_z.fill(pt=pt_z[num_selection], cat=trigger_title, sample_name=sample, weight=wd[num_selection])
            
            hpt_fj.fill(pt=SDm[num_selection], cat=trigger_title, sample_name="todas", weight=wd[num_selection])
            hpt_z.fill(pt=SDm[num_selection], cat=trigger_title, sample_name="todas", weight=wd[num_selection])




In [143]:
import hist
import numpy as np

# Define histogram axes and initialize histograms
ht_bins_fine = (25, 200, 2000)
zpt_axis = hist.axis.Regular(100, 0, 1000, name="pt", label=r"Z pT$^0$ [GeV]")
fjpt_axis = hist.axis.Regular(100, 200, 1000, name="pt", label=r"fj pT$^0$ [GeV]")
sdm_axis = hist.axis.Regular(100, 0, 1000, name="SDm", label=r"SD mass [GeV]")
cat_axis = hist.axis.StrCategory([], name="cat", growth=True)
sample_axis = hist.axis.StrCategory([], name="sample_name", growth=True)
process_axis = hist.axis.StrCategory([], name="process_name", growth=True)

hpt_fj = hist.Hist(fjpt_axis, cat_axis, sample_axis, process_axis)
hpt_z = hist.Hist(zpt_axis, cat_axis, sample_axis, process_axis)
hsd = hist.Hist(sdm_axis, cat_axis, sample_axis, process_axis)

# Fill histograms for each sample
for label, sample_dict in events_dict.items():
    for sample_name, events in sample_dict.items():
        pt_z = events["GenVPt"][0]
        pt_fj = events["ak8FatJetPt"][0]
        SDm_fj = events["ak8FatJetmsoftdrop"][0]
        wd = events["weight"][0]
        selection = SDm_fj > 40

        # Initialize histogram for this sample

        # Fill histograms for the sample
        hpt_fj.fill(pt=pt_fj[selection], cat="denominator", sample_name=sample_name, process_name=label, weight=wd[selection])
        hpt_z.fill(pt=pt_z[selection], cat="denominator", sample_name=sample_name, process_name=label, weight=wd[selection])
        hsd.fill(SDm=SDm_fj[selection], cat="denominator", sample_name=sample_name, process_name=label, weight=wd[selection])

        # Fill histograms for 'todas' category
        hpt_fj.fill(pt=pt_fj[selection], cat="denominator", sample_name=label, process_name=label, weight=wd[selection])
        hpt_z.fill(pt=pt_z[selection], cat="denominator", sample_name=label, process_name=label, weight=wd[selection])
        hsd.fill(SDm=SDm_fj[selection], cat="denominator", sample_name=label, process_name=label, weight=wd[selection])

        # Fill histograms for each trigger category
        for trigger_title, (triggers, trigger_label) in trigger_dict.items():
            trigger_selection = np.zeros_like(selection, dtype=bool)
            for hlt in triggers:
                trigger_selection |= (events[hlt].values == 1).squeeze()
            num_selection = trigger_selection & selection

            hpt_fj.fill(pt=pt_fj[num_selection], cat=trigger_title, sample_name=sample_name, process_name=label, weight=wd[num_selection])
            hpt_z.fill(pt=pt_z[num_selection], cat=trigger_title, sample_name=sample_name, process_name=label, weight=wd[num_selection])
            hsd.fill(SDm=SDm_fj[num_selection], cat=trigger_title, sample_name=sample_name, process_name=label, weight=wd[num_selection])
            
            hpt_fj.fill(pt=pt_fj[num_selection], cat=trigger_title, sample_name=label, process_name=label, weight=wd[num_selection])
            hpt_z.fill(pt=pt_z[num_selection], cat=trigger_title, sample_name=label, process_name=label, weight=wd[num_selection])
            hsd.fill(SDm=SDm_fj[num_selection], cat=trigger_title, sample_name=label, process_name=label, weight=wd[num_selection])


In [144]:
import matplotlib.pyplot as plt
import hist

def plot_den(hist_to_plot, output_dir, var):
    """
    Plot denominator histograms and save them to files.
    
    Args:
        hist_to_plot (dict): Dictionary of histograms to plot.
        output_dir (str): Directory where the plots will be saved.
    """
    for label, selector in samples.items():
        # Plot for each sample in selector
        fig, ax = plt.subplots()
        for sample in selector:
            hep.histplot(hist_to_plot[{"cat": "denominator", "sample_name": sample, "process_name": label}], ax=ax, label=sample)
        ax.legend()
        plt.title(f"Denominator Histograms for {label}")
        #plt.xlabel('Bins')
        plt.ylabel('Counts')
        plt.savefig(f"{output_dir}/{var}_denominator_{label}.png")  # Save the plot as a PNG file
        plt.close(fig)  # Close the figure to free up memory

        # Plot for the aggregate label
        fig, ax = plt.subplots()
        hep.histplot(hist_to_plot[{"cat": "denominator", "sample_name": label, "process_name": label}], ax=ax, label=label)
        ax.legend()
        plt.title(f"Denominator Histograms for All Samples - {label}")
        #plt.xlabel('Bins')
        plt.ylabel('Counts')
        plt.savefig(f"{output_dir}/{var}_denominator_all_{label}.png")  # Save the plot as a PNG file
        plt.close(fig)  # Close the figure to free up memory

# Define the output directory where plots will be saved
output_dir = "plots"

# Create the output directory if it does not exist
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)




In [ ]:
# Generate and save plots
plot_den(hsd, output_dir, var="SDm")
plot_den(hpt_fj, output_dir, var="fjpt")
plot_den(hpt_z, output_dir, var="zpt")

In [88]:
print(samples.items())

dict_items([('Zto2Q-4Jets', ['Zto2Q-4Jets_HT-400to600', 'Zto2Q-4Jets_HT-600to800', 'Zto2Q-4Jets_HT-800', 'Zto2Q-4Jets_HT-200to400']), ('Zto2Q-2Jets', ['Zto2Q-2Jets_PTQQ-100to200_1J', 'Zto2Q-2Jets_PTQQ-200to400_1J', 'Zto2Q-2Jets_PTQQ-400to600_1J', 'Zto2Q-2Jets_PTQQ-600_1J', 'Zto2Q-2Jets_PTQQ-100to200_2J', 'Zto2Q-2Jets_PTQQ-200to400_2J', 'Zto2Q-2Jets_PTQQ-400to600_2J', 'Zto2Q-2Jets_PTQQ-600_2J'])])


In [145]:
import matplotlib.pyplot as plt

def plot_1d(hist_to_plot, sample, sample_label, output_dir, var):
    """
    Plot 1D histograms and save them to files.
    
    Args:
        hist_to_plot (dict): Dictionary of histograms to plot.
        sample (str): Sample name.
        sample_label (str): Label for the sample.
        output_dir (str): Directory where the plots will be saved.
    """
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    for trigger_title, (triggers, trigger_label) in trigger_dict.items():
        
        den = hist_to_plot[{"cat": "denominator", "sample_name": sample, "process_name": sample_label}]
        num = hist_to_plot[{"cat": trigger_title, "sample_name": sample, "process_name": sample_label}]
        hep.histplot(
            num / den,
            yerr=abs(clopper_pearson_interval(num.view(), den.view()) - num.view() / den.view()),
            label=triggers[0],
            ax=ax,
            flow="none",
        )
    ax.legend()
    #leg = ax.legend(fontsize=8, bbox_to_anchor=(1.05, 1), loc="upper left")
    ax.set_ylabel("Signal Trig. Eff.")
    ax.set_ylim(0, 1)
    plt.title(f"Signal Trigger Efficiency for {sample}")
    
    # Save the plot
    plt.savefig(f"{output_dir}/{var}_signal_trigger_efficiency_{sample}.png")
    plt.close(fig)

# Define the output directory where plots will be saved
output_dir = "plots"

# Create the output directory if it does not exist
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Generate and save plots
for label, selector in samples.items():
    #for sample in selector:
    plot_1d(hpt_fj, label, label, output_dir, var="fjpt")
    plot_1d(hpt_z, label, label, output_dir, var="zpt")
    plot_1d(hsd, label, label, output_dir, var="SDm")


/var/folders/5l/g72gr7pn1qjfpv9ts3r8m3x80000gn/T/ipykernel_60870/618926411.py:20: RuntimeWarning: invalid value encountered in divide
  yerr=abs(clopper_pearson_interval(num.view(), den.view()) - num.view() / den.view()),
/var/folders/5l/g72gr7pn1qjfpv9ts3r8m3x80000gn/T/ipykernel_60870/618926411.py:20: RuntimeWarning: invalid value encountered in divide
  yerr=abs(clopper_pearson_interval(num.view(), den.view()) - num.view() / den.view()),
/var/folders/5l/g72gr7pn1qjfpv9ts3r8m3x80000gn/T/ipykernel_60870/618926411.py:20: RuntimeWarning: invalid value encountered in divide
  yerr=abs(clopper_pearson_interval(num.view(), den.view()) - num.view() / den.view()),
/var/folders/5l/g72gr7pn1qjfpv9ts3r8m3x80000gn/T/ipykernel_60870/618926411.py:20: RuntimeWarning: invalid value encountered in divide
  yerr=abs(clopper_pearson_interval(num.view(), den.view()) - num.view() / den.view()),
/var/folders/5l/g72gr7pn1qjfpv9ts3r8m3x80000gn/T/ipykernel_60870/618926411.py:20: RuntimeWarning: invalid value 

# 2D hist of FatJetPt and SoftDropMass


In [ ]:
import hist
import matplotlib.pyplot as plt
import numpy as np

# Define the axes
fjpt_axis = hist.axis.Regular(100, 200, 1000, name="pt", label=r"fj pT$^0$ [GeV]")
SDm_axis = hist.axis.Regular(100, 0, 300, name="SDm", label="Soft Drop Mass [GeV]")
cat_axis = hist.axis.StrCategory([], name="cat", growth=True)
sample_axis = hist.axis.StrCategory([], name="sample_name", growth=True)

# Create the histogram
h2d = hist.Hist(fjpt_axis, SDm_axis, cat_axis, sample_axis)

In [ ]:
# Filling the histogram

for sample in events_dict.keys():
    events = events_dict[sample][0]
    #pt_0 = events["GenVPt"][0]
    pt_0 = events["ak8FatJetPt"][0]
    SDm_0 = events["ak8FatJetmsoftdrop"][0]
    wd = events["weight"][0]
    #selection = (pt_0 > 0)
    selection = SDm_0 > 40

    # For separated samples
    #h2d.fill(pt=pt_0[selection], cat="denominator", sample_name=sample, weight=wd[selection])
    #h2d.fill(pt=pt_0, SDm=SDm_0 cat="denominator", sample_name=sample) #no weight or cuts

    #for all samples
    h2d.fill(pt=pt_0, SDm=SDm_0, cat="denominator", sample_name="todas") #no weight or cuts
    for trigger_title, (triggers, trigger_label) in trigger_dict.items():
        trigger_selection = np.zeros_like(selection)
        for hlt in triggers:
            trigger_selection |= (events[hlt].values == 1).squeeze()
        num_selection = trigger_selection
        #hpt.fill(pt=pt_0[num_selection & selection], cat=trigger_title, sample_name=sample, weight=wd[num_selection & selection])
        #hpt.fill(pt=pt_0[num_selection & selection], cat=trigger_title, sample_name="todas", weight=wd[num_selection & selection])

        #h2d.fill(pt=pt_0[num_selection], SDm=SDm_0[num_selection], cat=trigger_title, sample_name=sample)
        h2d.fill(pt=pt_0[num_selection], SDm=SDm_0[num_selection], cat=trigger_title, sample_name="todas")


In [ ]:
# Plot the 2D histogram
fig, ax = plt.subplots()
hep.hist2dplot(h2d[{"cat": "denominator", "sample_name": "todas"}], ax=ax)
#ax.set_xlabel("X-axis Label [units]")
#ax.set_ylabel("Y-axis Label [units]")
plt.show()